# Análise OD-2023

As seguintes análises foram feitas:
- Análise comparativa dos meios de transporte da OD 23 (quais os meios mais/menos utilizados) 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from utils.models import meios_transporte, criterio_br

In [ ]:
data_path = "../data/OD/OD-2023/od23_all.csv"
df = pd.read_csv(data_path)
# df.columns

In [ ]:
df.head()

## Comparação dos meios de transporte (OD-2023)

Colunas importantes:
- FE_PESS: fator de expansão da pessoa
- MODOPRIN: modo principal de viagem da pessoa

In [ ]:
print(meios_transporte)

In [ ]:
meios_fatorexp = df.groupby('MODOPRIN')['FE_PESS'].sum()
meios_fatorexp = meios_fatorexp.sort_values()

In [ ]:
meios_fatorexp.index = meios_fatorexp.index.map(lambda x: meios_transporte.get(x, "Desconhecido"))

In [ ]:
plt.figure(figsize=(12, 8))
meios_fatorexp.plot(kind='barh', color='skyblue', edgecolor='black')

plt.xlabel('Soma de FE_PESS (em milhões)')
plt.ylabel('Meios de Transporte')
plt.title('Distribuição de FE_PESS por Meio de Transporte')

for index, value in enumerate(meios_fatorexp):
    plt.text(value, index, f'{value/1e6:.2f}M', va='center')  

plt.xticks(ticks=plt.xticks()[0], labels=[f'{int(x/1e6)}M' for x in plt.xticks()[0]])

plt.tight_layout()  
plt.show()

In [ ]:
meios_fatorexp_normalizado = (meios_fatorexp / meios_fatorexp.sum()) * 100

plt.figure(figsize=(12, 8))
meios_fatorexp_normalizado.plot(kind='barh', color='skyblue', edgecolor='black')

plt.xlabel('Percentual (%)')
plt.ylabel('Meios de Transporte')
plt.title('Distribuição Percentual de FE_PESS por Meio de Transporte')

for index, value in enumerate(meios_fatorexp_normalizado):
    plt.text(value + 0.1, index, f'{value:.1f}%', va='center')

plt.tight_layout()
plt.show()

### Análise comparativa percentual entre os meios de transporte

Com os gráficos acima, observa-se que os meios de transporte mais utilizados são, em ordem decrescente: a pé, dirigindo automóvel, e ônibus(e derivados) do município de São Paulo.

- A pé - 27.7%
- Dirigindo automóvel - 21.0% 
- Ônibus(e derivados) do município de São Paulo - 10.7%

## Caracterização do Perfil do Pedestre

Colunas importantes:
- FE_PESS: fator expansão da pessoa ('peso da linha')
- MODOPRIN: modo principal

01: Metrô, 02: Trem, 03:Monotrilho, 04: Ônibus/micro-ônibus/van do município de São Paulo, 05: Ônibus/micro-ônibus/van de outros municípios, 06: Ônibus/micro-ônibus/van metropolitano, 07: Transporte Fretado, 08: Transporte Escolar, 09: Dirigindo Automóvel, 10: Passageiro de Automóvel, 11: Táxi Convencional, 12: Táxi não Convencional / aplicativo, 13: Dirigindo Moto, 14: Passageiro de Moto, 15: Passageiro de Mototáxi, 16: Bicicleta, 17: A Pé, 18: Outros

- IDADE
- SEXO

1: Masculino, 2: Feminino, 3: Não respondeu


- ESTUDA: estuda atualmente

1: Não, 2: Creche/Pré-Escola, 3: 1º Grau /Fundamental, 4: 2º Grau/Médio, 5: Superior/Universitário, 6: Outros

- GRAU_INS: grau de instrução

1: Não Alfabetizado/Fundamental I Incompleto, 2: Fundamental I Completo/Fundamental II Incompleto, 3: Fundamental II Completo/Médio Incompleto, 4: Médio Completo/Superior Incompleto, 5: Superior Completo

- CD_ATIVI: condição atividade (trabalho regular, bico...)

1: Tem trabalho regular, 2: Faz bico, 3: Em Licença Médica, 4: Aposentado/Pensionista, 5: Sem Trabalho, 6: Nunca Trabalhou, 7: Dona de Casa, 8: Estudante

- VL_REN_I: valor da renda individual

- MOTIVO_D: motivo no destino

1: Trabalho Indústria, 2: Trabalho Comércio, 3: Trabalho Serviços, 4: Escola/Educação, 5: Compras, 6: Médico/Dentista/Saúde, 7: Recreação/Visitas/Lazer, 8: Residência, 9: Procurar Emprego, 10: Assuntos Pessoais, 11: Refeição

- PE_BICI: por que viajou a pé ou bicicleta

1: Pequena distância, 2: Condução cara, 3: Ponto/Estação distante, 4: Condução demora para passar, 5: Viagem demorada, 6: Condução lotada, 7: Atividade física, 8: Medo de contágio, 9: Outros motivos

- DISTANCIA: distância da viagem em metros

- CRITERIOBR: Critério de Classificação Econômica Brasil

1: A, 2: B1, 3: B2, 4: C1, 5: C2, 6:  D e E

Correlação entre variáveis

In [ ]:
df[['IDADE', 'VL_REN_I', 'DISTANCIA', 'FE_PESS']].corr(method='pearson')

In [ ]:
df[['IDADE', 'VL_REN_I', 'DISTANCIA', 'FE_PESS']].corr(method='spearman')

Tabelas de contingência entre sexo e modo de transporte

In [ ]:
pd.crosstab(df['SEXO'], df['MODOPRIN'], normalize='index')

Definição do Objeto de Estudo - pedestres

In [ ]:
# Filtrando pedestres
pedestrians = df.query('MODOPRIN == 17')

### Pré-processamento dos Dados
Criação de categorias: Faixas etárias e Distâncias percorridas

Categorização do Criterio BR e Sexo

In [ ]:
# Categorização da distância (em metros)
bins_distancia = [0, 500, 1000, 2000, 5000, float('inf')]
labels_distancia = ['Muito curta (<500m)', 'Curta (500-1000m)', 'Média (1001-2000m)', 
                   'Longa (2001-5000m)', 'Muito longa (>5000m)']

pedestrians['CAT_DISTANCIA'] = pd.cut(pedestrians['DISTANCIA'], bins=bins_distancia, labels=labels_distancia)

In [ ]:
# Categorização de idade
bins_idade = [0, 12, 18, 30, 50, 65, 120]
labels_idade = ['Criança (0-12)', 'Adolescente (13-18)', 'Jovem (19-30)', 
               'Adulto (31-50)', 'Adulto 50+ (51-65)', 'Idoso (65+)']

pedestrians['CAT_IDADE'] = pd.cut(pedestrians['IDADE'], bins=bins_idade, labels=labels_idade)

In [ ]:
# Mapeamento dos critérios econômicos
pedestrians['CLASSE_ECON'] = pedestrians['CRITERIOBR'].map(criterio_br)
pedestrians['CLASSE_ECON'].value_counts()

In [ ]:
# Mapeamento do sexo
sexo_labels = {1: 'Masculino', 2: 'Feminino'}
pedestrians['SEXO_LABEL'] = pedestrians['SEXO'].map(sexo_labels)

### Análise Descritiva Básica
**Perfil demográfico**
- Distribuição por sexo (SEXO)
- Distribuição por idade (IDADE)
- Distribuição por grau de instrução (GRAU_INS)

**Características socioeconômicas**
- Distribuição por condição de atividade (CD_ATIVI)
- Distribuição de renda (VL_REN_I)
- Relação entre renda e modo pedestre

In [ ]:
sexo_dist = pedestrians.groupby('SEXO_LABEL')['FE_PESS'].sum() 
# print("Distribuição por sexo (%):\n", sexo_dist)
sexo_dist.plot(kind='bar', title='Distribuição por Sexo (Pedestres)', ylabel='%')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
idade_stats = pedestrians['IDADE'].describe()
print("\nEstatísticas de idade (pedestres):\n", idade_stats)

plt.hist(pedestrians['IDADE'], bins=15, weights=pedestrians['FE_PESS'], edgecolor='black')
plt.title('Distribuição da Idade (Pedestres)')
plt.xlabel('Idade')
plt.ylabel('Número expandido de pessoas')
plt.tight_layout()
plt.show()

In [ ]:
grau_labels = {
    1: 'Fundamental I Incompleto',
    2: 'Fundamental I Comp. / Fund II Incomp.',
    3: 'Fundamental II Comp. / Médio Incomp.',
    4: 'Médio Comp. / Sup. Incompleto',
    5: 'Superior Completo'
}
pedestrians['GRAU_LABEL'] = pedestrians['GRAU_INS'].map(grau_labels)

grau_dist = pedestrians.groupby('GRAU_LABEL')['FE_PESS'].sum().sort_values(ascending=False)
grau_dist.plot(kind='barh', title='Grau de Instrução dos Pedestres', xlabel='Pessoas (expandidas)')
plt.tight_layout()
plt.show()


In [ ]:
atividade_labels = {
    1: 'Trabalho regular',
    2: 'Bico',
    3: 'Licença médica',
    4: 'Aposentado',
    5: 'Sem trabalho',
    6: 'Nunca trabalhou',
    7: 'Dona de casa',
    8: 'Estudante'
}
pedestrians['ATIVI_LABEL'] = pedestrians['CD_ATIVI'].map(atividade_labels)

ativi_dist = pedestrians.groupby('ATIVI_LABEL')['FE_PESS'].sum().sort_values(ascending=False)
ativi_dist.plot(kind='bar', title='Condição de Atividade dos Pedestres', ylabel='Pessoas (expandidas)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
renda_stats = pedestrians['VL_REN_I'].describe()
# print("\nEstatísticas da renda individual (pedestres):\n", renda_stats)

plt.figure(figsize=(8, 5))
sns.boxplot(data=pedestrians, x='VL_REN_I')
plt.title('Boxplot da Renda Individual dos Pedestres')
plt.xlabel('Renda (R$)')
plt.tight_layout()
plt.show()

In [ ]:
renda_grau = pedestrians.groupby('GRAU_LABEL')['VL_REN_I'].mean().sort_values()
renda_grau.plot(kind='barh', title='Renda Média por Grau de Instrução (Pedestres)', xlabel='Renda Média (R$)')
plt.tight_layout()
plt.show()

### Análise Específica do Comportamento como Pedestre
**Motivações**
- Razões para caminhar (PE_BICI)
- Motivos de viagem (MOTIVO_D)

**Padrões de deslocamento**
- Distribuição de distâncias percorridas (DISTANCIA)
- Relação entre distância e motivo da viagem
- Relação entre distância e características demográficas